In [24]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-aj1z3ufr
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-aj1z3ufr
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=594db48995349fd54a4d733b163d40c2e0e41f3c4262d31765e16407f5cc92cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-r1teuxe9/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


1) Write a CUDA program to compute the sum of an array of elements. Input:
Number of elements in the array. Output: Array sum (Print only once).

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__managed__ int N = 10;

#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )
static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

__global__ void sumOfArray(int *a, int *sum) 
{
    unsigned short tid = threadIdx.x;
    if(tid < N)
        atomicAdd(sum, a[tid]);
}

int main() 
{
    
    int *arr, sum = 0;
    int *da, *ds;
    arr = (int*)malloc(N*sizeof(int));

    HANDLE_ERROR( cudaMalloc((void **)&da, N*sizeof(int)) );
    HANDLE_ERROR( cudaMalloc((void **)&ds, sizeof(int)) );
    
    for(int i = 0;i<N;i++){
        arr[i] = i+1;
    }
 
    HANDLE_ERROR (cudaMemcpy (da , arr , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (ds , &sum , sizeof(int) , cudaMemcpyHostToDevice));

    sumOfArray<<<1, N>>>(da, ds);

    HANDLE_ERROR (cudaMemcpy (&sum , ds , sizeof(int) , cudaMemcpyDeviceToHost));
 
    printf("sum = %d\n", sum);
    
    return 0;
}


sum = 55



2) Write a CUDA program to compute the sum of two arrays. Input: Number of
elements in the array. Output: Array of sums (Print only once).

In [ ]:
%%cu
#include <stdio.h>

#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )
static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

__managed__ int N = 100 ;

__global__ void Vector_Addition ( const int *dev_a , const int *dev_b , int *dev_c)
{
      unsigned short tid = threadIdx.x ;
      if ( tid < N ) 
            dev_c [tid] = dev_a[tid] + dev_b[tid] ;
}

int main (void)
{
      int *Host_a, *Host_b, *Host_c;
      Host_a = (int*)malloc(N*sizeof(int));
      Host_b = (int*)malloc(N*sizeof(int));
      Host_c = (int*)malloc(N*sizeof(int));

      int *dev_a, *dev_b, *dev_c ;

      HANDLE_ERROR ( cudaMalloc((void **)&dev_a , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_b , N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_c , N*sizeof(int) ) );

      for ( int i = 0; i <N ; i++ )
      {
            Host_a[i] = -i ;
            Host_b[i] = i*i ; 
      }
      HANDLE_ERROR (cudaMemcpy (dev_a , Host_a , N*sizeof(int) , cudaMemcpyHostToDevice));
      HANDLE_ERROR (cudaMemcpy (dev_b , Host_b , N*sizeof(int) , cudaMemcpyHostToDevice));

      Vector_Addition <<< 1, N  >>> (dev_a , dev_b , dev_c ) ;

      HANDLE_ERROR (cudaMemcpy(Host_c , dev_c , N*sizeof(int) , cudaMemcpyDeviceToHost));

      for ( int i = 0; i<N; i++ )
      printf ("%d + %d = %d\n", Host_a[i] , Host_b[i] , Host_c[i] ) ;

      cudaFree (dev_a) ;
      cudaFree (dev_b) ;
      cudaFree (dev_c) ;
 
      free(Host_a);
      free(Host_b);
      free(Host_c);

      return 0 ;

}

0 + 0 = 0
-1 + 1 = 0
-2 + 4 = 2
-3 + 9 = 6
-4 + 16 = 12
-5 + 25 = 20
-6 + 36 = 30
-7 + 49 = 42
-8 + 64 = 56
-9 + 81 = 72
-10 + 100 = 90
-11 + 121 = 110
-12 + 144 = 132
-13 + 169 = 156
-14 + 196 = 182
-15 + 225 = 210
-16 + 256 = 240
-17 + 289 = 272
-18 + 324 = 306
-19 + 361 = 342
-20 + 400 = 380
-21 + 441 = 420
-22 + 484 = 462
-23 + 529 = 506
-24 + 576 = 552
-25 + 625 = 600
-26 + 676 = 650
-27 + 729 = 702
-28 + 784 = 756
-29 + 841 = 812
-30 + 900 = 870
-31 + 961 = 930
-32 + 1024 = 992
-33 + 1089 = 1056
-34 + 1156 = 1122
-35 + 1225 = 1190
-36 + 1296 = 1260
-37 + 1369 = 1332
-38 + 1444 = 1406
-39 + 1521 = 1482
-40 + 1600 = 1560
-41 + 1681 = 1640
-42 + 1764 = 1722
-43 + 1849 = 1806
-44 + 1936 = 1892
-45 + 2025 = 1980
-46 + 2116 = 2070
-47 + 2209 = 2162
-48 + 2304 = 2256
-49 + 2401 = 2352
-50 + 2500 = 2450
-51 + 2601 = 2550
-52 + 2704 = 2652
-53 + 2809 = 2756
-54 + 2916 = 2862
-55 + 3025 = 2970
-56 + 3136 = 3080
-57 + 3249 = 3192
-58 + 3364 = 3306
-59 + 3481 = 3422
-60 + 3600 = 3540
-61 + 3

3) Write a CUDA program to multiply two matrices. Input: Matrix 1 size: m X n
Matrix 2 size: n X p. Output: Result matrix (Print only once).

In [33]:
%%cu
#include <stdio.h>

#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )
static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

__managed__ int M = 200;
__managed__ int N = 200 ;
__managed__ int O = 200 ;

__global__ void matrixMul ( const int *dev_a , const int *dev_b , int *dev_c)
{
      unsigned short j = threadIdx.x ;
      unsigned short i = blockIdx.x ;
      if (i<M && j<O) {
          dev_c[i*N + j] = 0;
          for(int k = 0; k<N;k++){
              dev_c[i*O+j] += dev_a[i*N+k] * dev_b[k*O+j];
          }
      }
}

int main (void)
{

      int *Host_a, *Host_b, *Host_c;
      Host_a = (int*)malloc(M*N*sizeof(int));
      Host_b = (int*)malloc(N*O*sizeof(int));
      Host_c = (int*)malloc(M*O*sizeof(int));

      int *dev_a, *dev_b, *dev_c ;

      HANDLE_ERROR ( cudaMalloc((void **)&dev_a , M*N*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_b , N*O*sizeof(int) ) );
      HANDLE_ERROR ( cudaMalloc((void **)&dev_c , M*O*sizeof(int) ) );

      for(int i=0; i<M; i++)
          for (int j=0; j<N; j++)
              Host_a[i*N + j] = i+j;
      for(int i=0; i<N; i++)
          for (int j=0; j<O; j++)
              Host_b[i*O + j] = i+j;
            
      HANDLE_ERROR (cudaMemcpy (dev_a , Host_a , M*N*sizeof(int) , cudaMemcpyHostToDevice));
      HANDLE_ERROR (cudaMemcpy (dev_b , Host_b , N*O*sizeof(int) , cudaMemcpyHostToDevice));

      matrixMul <<< M, O  >>> (dev_a , dev_b , dev_c ) ;

      HANDLE_ERROR (cudaMemcpy(Host_c , dev_c , M*O*sizeof(int) , cudaMemcpyDeviceToHost));
      
      printf("A\n");
      for(int i=0; i<M; i++){
          for (int j=0; j<N; j++)
              printf("%d ", Host_a[i*N + j]);
          printf("\n");

      }
      printf("B\n");
      for(int i=0; i<N; i++){
          for (int j=0; j<O; j++)
              printf("%d ", Host_b[i*O + j]);
          printf("\n");

      }
      printf("C\n");
      for(int i=0; i<M; i++){
          for (int j=0; j<O; j++)
              printf("%d ", Host_c[i*O + j]);
          printf("\n");

      }


      cudaFree (dev_a) ;
      cudaFree (dev_b) ;
      cudaFree (dev_c) ;
 
      free(Host_a);
      free(Host_b);
      free(Host_c);

      return 0 ;

}

A
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104

4) Write a report (1-page) on CUDA libraries
(https://developer.nvidia.com/gpu-accelerated-libraries).

**REPORT ON NVIDIA CUDA-X**

**INTRODUCTION:**
	NVIDIA CUDA-X is a collection of GPU-accelerated libraries, tools, and technologies built on top of NVIDIA CUDA, that delivers higher performance when compared to the CPU alternatives across multiple application domains, from Artificial Intelligence(AI) to High Performance Computing(HPC). NVIDIA libraries provide an easiest way to get started with GPU acceleration, whether for building a new application or accelerating an existing application.

**MATH LIBRARIES:**
	GPU-accelerated math libraries lay the foundation for compute-intensive applications in areas such as molecular dynamics, computational fluid dynamics, computational chemistry, medical imaging, and seismic exploration. Some of the math libraries are cuBLAS(BLAS library), cuFFT(library for Fast Fourier Transforms),  CUDA Math Library (standard mathematical functions), cuRAND(random number generation), cuSOLVER(dense and sparse direct solvers), cuSPARSE(BLAS for sparse matrices), cuTENSOR (tensor library algebra library), AmgX(linear solvers for simulations and implicit unstructured methods).

**PARALLEL ALGORITHM LIBRARIES:**
	GPU-accelerated libraries of highly efficient parallel algorithms for several operations in C++ and for use with graphs when studying relationships in natural sciences, logistics, travel planning, and more. Thrust is a powerful library of parallel algorithms and data structures. Thrust provides a flexible, high-level interface for GPU programming that greatly enhances developer productivity.

 **IMAGE AND VIDEO LIBRARIES:**
	GPU-accelerated libraries for image and video decoding, encoding, and processing that leverage CUDA and specialized hardware components of GPUs. Some of the libraries are: nvJPEG (High performance library for JPEG decoding), NVIDIA Performance Primitives (image, video and signal processing functions), NVIDIA Video Codec SDK (hardware-accelerated video encode and decode on Windows and Linux), NVIDIA Optical Flow SDK (dedicated to computing the relative motion of pixels between images). 

**COMMUNICATION LIBRARIES:**
	Performance-optimized multi-GPU and multi-node communication primitives. Some of the NVIDIA provided libraries are: NVSHMEM (OpenSHMEM standard for GPU memory), NCCL (Open-source library for fast multi-GPU, multi-node communications that maximizes bandwidth while maintaining low latency.

**DEEP LEARNING LIBRARIES:**
	GPU-accelerated libraries for Deep Learning applications that leverage CUDA and specialized hardware components of GPUs. Some of those libraries include: NVIDIA cuDNN (library of primitives for deep neural networks), NVIDIA TensorRT(high-performance deep learning inference optimizer and runtime for production deployment), NVIDIA Jarvis (platform for developing engaging and contextual AI-powered conversation apps), NVIDIA DeepStream SDK (real-time streaming analytics toolkit for AI-based video understanding and multi-sensor processing), NVIDIA DALI(portable, open-source library for decoding and augmenting images and videos to accelerate deep learning applications).

**PARTNER LIBRARIES:**
	NVIDIA has some libraries that are not completely developed by NVIDIA but have contributed to those libraries. These include OpenCV, FFmpeg, ArrayFire, MAGMA, IMSL Fortran Numerical Library, Gunrock, CHOLMOD, Triton Ocean SDK, CUVIlib.


5) Refer to the attached paper. Implement BFS on CUDA. Either generate a random
graph or download a graph from SNAP (example: http://snap.stanford.edu/data/com-Orkut.html)

In [16]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )
static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

__managed__ int N = 10 ;
__managed__ int E = 20 ;   //#In case of undirected graph 2*Edges
int s = 2;    //#Source vertex (vertex starts from 0)

__global__ void bfs_kernel(int *v, int *a, bool *f, bool *x, int *c) {
    unsigned int tid = threadIdx.x;
    if(tid < N) {
        if(f[tid]){
            f[tid] = false;
            x[tid] = true;
            int st = v[tid], end = tid+1 < N ? v[tid+1] : E;
            while(st < end) {
                if(!x[a[st]]){
                    f[a[st]] = true;  
                    c[a[st]] = c[tid] + 1;
                }
                st++;
            }
        }
    }
}

void getRandomGeneratedMatrix (int *a, int *v) {
      int *adjMat = (int*)calloc(N*N, sizeof(int));
      int i = 0, j,k;
      while(i<E) {
          j = rand()%N; k = rand()%N;
          if(adjMat[j*N+k] == 0) {
              adjMat[j*N+k] = 1;
              adjMat[k*N+j] = 1;
              i+=2;
          }
      }
      i = 0;
      for(j=0;j<N;j++) {
          v[j] = i;
          for(k=0;k<N;k++){
              if(adjMat[j*N + k]) {
                  a[i++] = k;
              }
          }
      }
}

int main(void) {
    int *v, *a, *c;
    bool *f, *x;
    v = (int*) malloc(N*sizeof(int));
    a = (int*) malloc(E*sizeof(int));
    c = (int*) malloc(N*sizeof(int));
    f = (bool*) malloc(N*sizeof(bool));
    x = (bool*) malloc(N*sizeof(bool));

    getRandomGeneratedMatrix(a, v);

    printf("Vertce array\n");
    for(int i=0;i<N;i++)
        printf("%d ", v[i]);
    printf("\nAdjancy array \n");
    for(int i=0;i<E;i++)
        printf("%d ", a[i]);

    int *da, *dv, *dc;
    bool *df, *dx;

    HANDLE_ERROR ( cudaMalloc((void **)&dv , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&da , E*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dc , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&df , N*sizeof(bool) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dx , N*sizeof(bool) ) );

    for(int i=0;i<N;i++){
      c[i] = -1;
      f[i] = false;
      x[i] = false;
    }

    f[s] = true;
    c[s] = 0;
    bool fvi = true;

    HANDLE_ERROR (cudaMemcpy (da , a , E*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (dv , v , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (dc , c , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (df , f , N*sizeof(bool) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (dx , x , N*sizeof(bool) , cudaMemcpyHostToDevice));

    while(fvi) {
        bfs_kernel<<<1, N>>> (dv, da, df, dx, dc);
        HANDLE_ERROR (cudaMemcpy (f , df , N*sizeof(bool) , cudaMemcpyDeviceToHost));
        fvi = false;
        for(int i=0;i<N;i++)
            if(f[i]){
                fvi = true;
                break;
            }
    }

    HANDLE_ERROR (cudaMemcpy (c , dc , N*sizeof(int) , cudaMemcpyDeviceToHost));
    printf("\nBFS Results \n");
    for(int i=0;i<N;i++) {
        printf("%d ", c[i]);
    }

    printf("\nNote: -1 represents infinity");
    return 0;
}

Vertce array
0 2 4 6 8 8 10 13 16 17 
Adjancy array 
6 9 8 9 6 7 5 6 3 7 0 2 3 2 5 9 1 0 1 7 
BFS Results 
2 3 0 2 -1 2 1 1 4 2 
Note: -1 represents infinity


6) Refer to the attached paper. Implement SSSP on CUDA. Either generate a
random graph or download a graph from SNAP (example: http://snap.stanford.edu/data/com-Orkut.html)

In [22]:
%%cu
#include <stdio.h>
#include <limits.h>

#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )
static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

__managed__ int N = 10 ;
__managed__ int E = 20 ; // No. of edges * 2

__global__ void sssp_kernel1(int *v, int *a, int *w, bool *m, int *c, int *u) {
    unsigned int tid = threadIdx.x;
    if(tid < N) {
        if(m[tid]){
            m[tid] = false;
            int st = v[tid], end = tid+1 < N ? v[tid+1] : E;
            while(st < end) {
                atomicMin(&u[a[st]], c[tid] + w[st]);
                st++;
            }
        }
    }
}

__global__ void sssp_kernel2(int *v, int *a, int *w, bool *m, int *c, int *u) {
    unsigned int tid = threadIdx.x;
    if(tid < N) {
        if (c[tid] > u[tid]) {
            c[tid] = u[tid];
            m[tid] = true;
        }
        u[tid] = c[tid];
    }
}

void getRandomGeneratedMatrix (int *a, int *v, int *w) {
      int *adjMat = (int*)calloc(N*N, sizeof(int));
      int i = 0, j,k;
      while(i<E) {
          j = rand()%N; k = rand()%N;
          if(adjMat[j*N+k] == 0) {
              adjMat[j*N+k] = rand()%200 + 1;
              adjMat[k*N+j] = adjMat[j*N+k];
              i+=2;
          }
      }
      i = 0;
      for(j=0;j<N;j++) {
          v[j] = i;
          for(k=0;k<N;k++){
              if(adjMat[j*N + k]) {
                  a[i] = k;
                  w[i++] = adjMat[j*N + k];
              }
          }
      }
}

int main(void) {
    int *v, *a;
    int *w, *c, *u;
    bool *m;
    v = (int*) malloc(N*sizeof(int));
    a = (int*) malloc(E*sizeof(int));
    c = (int*) malloc(N*sizeof(int));
    w = (int*) malloc(E*sizeof(int));
    m = (bool*) malloc(N*sizeof(bool));
    u = (int*) malloc(N*sizeof(int));

    getRandomGeneratedMatrix(a,v,w);
    
    printf("Vertce array\n");
    for(int i=0;i<N;i++)
        printf("%d ", v[i]);
    printf("\nAdjancy array \n");
    for(int i=0;i<E;i++)
        printf("%d ", a[i]);
    printf("\nAdjancy weight array \n");
    for(int i=0;i<E;i++)
        printf("%d ", w[i]);

    int *da, *dv;
    int *dc, *dw, *du;
    bool *dm;

    HANDLE_ERROR ( cudaMalloc((void **)&dv , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&da , E*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dc , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dw , E*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&du , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dm , N*sizeof(bool) ) );

    for(int i=0;i<N;i++){
      c[i] = INT_MAX;
      m[i] = false;
      u[i] = INT_MAX;
    }

    int s = 0;
    m[s] = true;
    c[s] = 0;
    u[s] = 0;
    bool fvi = true;

    HANDLE_ERROR (cudaMemcpy (da , a , E*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (dv , v , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (dc , c , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (du , u , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (dw , w , E*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (dm , m , N*sizeof(bool) , cudaMemcpyHostToDevice));

    while(fvi) {
        sssp_kernel1<<<1, N>>> (dv, da, dw, dm, dc, du);
        
        HANDLE_ERROR (cudaMemcpy (u , du , N*sizeof(int) , cudaMemcpyDeviceToHost));
        sssp_kernel2<<<1, N>>> (dv, da, dw, dm, dc, du);
        HANDLE_ERROR (cudaMemcpy (m , dm , N*sizeof(bool) , cudaMemcpyDeviceToHost));
        fvi = false;
        for(int i=0;i<N;i++)
            if(m[i]){
                fvi = true;
                break;
            }
    }

    HANDLE_ERROR (cudaMemcpy (c , dc , N*sizeof(int) , cudaMemcpyDeviceToHost));
    printf("\nSSSP Array\n");
    for(int i=0;i<N;i++) {
        printf("%d ", c[i]);
    }

    printf("\n2147483647 is INFINITY");
    return 0;
}

Vertce array
0 2 4 8 11 11 13 16 16 17 
Adjancy array 
6 9 2 8 1 3 6 9 2 5 6 3 9 0 2 3 1 0 2 5 
Adjancy weight array 
27 164 28 168 28 68 50 131 68 136 178 136 3 27 50 178 168 164 131 3 
SSSP Array
0 105 77 145 2147483647 167 27 2147483647 273 164 
2147483647 is INFINITY


7) Use Thrust library to sort a vector of elements.

In [ ]:
%%cu
#include<thrust/host_vector.h>
#include<thrust/device_vector.h>
#include<thrust/generate.h>
#include<thrust/sort.h>
#include<thrust/copy.h>
#include<cstdlib>

int main(void) {
    thrust::host_vector<int> h_vec(100);
    thrust::generate(h_vec.begin(), h_vec.end(), rand);

    thrust::device_vector<int> d_vec = h_vec;

    thrust::sort(d_vec.begin(), d_vec.end());

    thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());

    for(int i=0;i<h_vec.size();i++) {
        printf("%d ", h_vec[i]);
    }

    return 0;
}

35005211 42999170 84353895 135497281 137806862 149798315 184803526 233665123 278722862 294702567 304089172 336465782 356426808 412776091 424238335 468703135 491705403 511702305 521595368 572660336 596516649 608413784 610515434 628175011 635723058 709393584 719885386 749241873 752392754 756898537 760313750 783368690 805750846 846930886 855636226 859484421 861021530 939819582 943947739 945117276 982906996 1025202362 1059961393 1100661313 1101513929 1102520059 1125898167 1129566413 1131176229 1141616124 1159126505 1189641421 1264095060 1303455736 1315634022 1350490027 1365180540 1369133069 1374344043 1411549676 1424268980 1433925857 1469348094 1474612399 1477171087 1540383426 1548233367 1585990364 1632621729 1649760492 1653377373 1656478042 1681692777 1714636915 1726956429 1734575198 1749698586 1780695788 1801979802 1804289383 1827336327 1843993368 1889947178 1911759956 1914544919 1918502651 1937477084 1956297539 1957747793 1967513926 1973594324 1984210012 1998898814 2001100545 2038664370